In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
model = SentenceTransformer(
    "all-MiniLM-L6-v2", device="cpu"
)

In [7]:
df = pd.read_json("startups_demo.json", lines=True)

In [11]:
vectors = model.encode(
    [row.alt + ". " + row.description for row in df[:100].itertuples()],
    show_progress_bar=True,
)

Batches: 100%|██████████| 4/4 [00:03<00:00,  1.22it/s]


In [13]:
vectors.shape

(100, 384)

In [14]:
np.save("startup_vectors.npy", vectors, allow_pickle=False)

In [15]:
# Import client library
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient("http://localhost:6333")

In [16]:
if not client.collection_exists("startups"):
    client.create_collection(
        collection_name="startups",
        vectors_config=VectorParams(size=384, distance=Distance.COSINE),
    )

In [ ]:
fd = open("startups_demo.json")

payload = map(json.loads, fd)


vectors = np.load("./startup_vectors.npy")

In [20]:
client.upload_collection(
    collection_name="startups",
    vectors=vectors,
    payload=payload,
    ids=None, 
    batch_size=256,
)